In [1]:
import pandas as pd
import numpy as np
import altair as alt
import os
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis


In [2]:
path_root_data = os.path.join ('..', 'Data', 'Extracted', 'Databases')
path_all_fasta = os.path.join (path_root_data, 'fasta_files', 'AllBins')



In [14]:
# There are 1364 MAGs enclosed in FASTA files
# I have to traverse every FASTA file end in each file every sequence

list_of_series = []

fasta_files = [i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')]

for fasta_file_name in fasta_files:
    with open(os.path.join(path_all_fasta, fasta_file_name), 'r') as input_file:
        #identifiers = []
        #lengths = []
        sequence = []
        for fasta_string in SeqIO.parse(input_file, "fasta"):
            #identifiers.append(seq_record.id)
            #lengths.append(len(seq_record.seq))
            sequence.append(str(fasta_string.seq))
        
        list_of_series.append(pd.Series(sequence))


In [20]:
# Memory usage after this is ~ 4.5GB
list_of_series[0]

0      AAAGCGGAACCCGTTGGCGGCAATGGTGGCACTTTCCATGTCCAGG...
1      ACGCCCCATGCATCGCCTGCGTCAAGCTGCCTTGTTGTTGATTTCG...
2      AAAGGCGCAGCCAGTCGCACAAACGCCAATAACCAATGAAATCCCC...
3      ATGACGCTCGTCGTCGACGGCTACAACAACGATCCACGCGAGCTGG...
4      AAGAGGTCAACATGTACGCGGTCATAAAAACCGGCGGCAAGCAGTA...
                             ...                        
321    GACGTTGAGTCGCCTCATTTACTACAGCTGACGTTCGGTGAGCGTA...
322    CGCATCAGCTCATAGCGGTCCATGGTCAGAGAGTCGCTGGCGCTAG...
323    AACCCGGTAACGCCCAGCAGGCCCAAGCCCACCACCAGCATGCCGG...
324    CCACCACGCGCGCCACCAGCGGGTGGCGCACGATATCCAAGCTGGT...
325    GGTCGTGGACGTCATGGTCTTCGCGATCTCGTATTCAGCTCACATC...
Length: 326, dtype: object